In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
import h5py
import wandb
import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)



2024-04-02 13:57:01.748624: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 13:57:01.780781: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 13:57:01.780806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 13:57:01.781670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-02 13:57:01.787147: I tensorflow/core/platform/cpu_feature_guar

In [6]:
import wandb
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras import layers, models, optimizers

# Ensure wandb is installed: pip install wandb
# Initialize your W&B project before running the script

# Dataset loading and preparation
def load_and_prepare_dataset(batch_size):
    with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
        images = np.array(hdf.get('images'))
        boxes = np.array(hdf.get('boxes'))
    

    image_normalized = (images + 1e-9) / 9.26
    normalized_boxes = boxes / [1, 64, 64, 64, 64]

    images_np = image_normalized
    probabilities = np.array(normalized_boxes[:, :, :-4])
    probabilities = tf.expand_dims(probabilities, axis=1)
    boxes_np = np.array(normalized_boxes[:, :, 1:])
    boxes_np = tf.expand_dims(boxes_np, axis=1)

    dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
    dataset = dataset.shuffle(buffer_size=8000).batch(batch_size)
    return dataset

# Model building
def build_model():
    input_shape = (64, 64, 1)
    num_classes = 280
    num_coordinates = 4

    x_input = layers.Input(shape=input_shape)
    #Layer 1
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 2
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 3
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 4
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    #Layer 5
    x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    
    x = layers.Flatten()(x)
    x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
    x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
    x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)
    x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)

    model = models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
    return model

# Optimizer selection
def select_optimizer(optimizer_name, learning_rate):
    if optimizer_name == 'adam':
        return optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        return optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer specified.")

# Training function with sweep
def train_with_sweep():
    
    with wandb.init():
        config = wandb.config
        # # config.run.id
        # print(wandb.run.id)
        # print(wandb.run.name)
        dataset = load_and_prepare_dataset(config.batch_size)
        model = build_model()
        
        optimizer = select_optimizer(config.optimizer, config.learning_rate)
        model.compile(optimizer=optimizer, 
                      loss={'x_prob_reshape': 'binary_crossentropy', 'x_boxes_reshape': 'mean_squared_error'}, 
                      metrics={'x_prob_reshape': 'accuracy'})

        # Fit the model
        model.fit(dataset, epochs=config.epochs, callbacks=[wandb.keras.WandbCallback()])
        model.save(f'{wandb.run.name}.h5')


In [7]:

sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
    
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 11}
    })
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        'values': [32, 64, 256]
      }
    })

In [8]:
sweep_id = wandb.sweep(sweep_config, project="Working Hyperparameter Tuning Yichen edit A2")
wandb.agent(sweep_id, train_with_sweep, count=5)

Create sweep with ID: 0h9lxtih
Sweep URL: https://wandb.ai/alphanium/Working%20Hyperparameter%20Tuning%20Yichen%20edit%20A2/sweeps/0h9lxtih


wandb: Agent Starting Run: s393sb1f with config:
wandb: 	batch_size: 256
wandb: 	epochs: 11
wandb: 	learning_rate: 0.0823033353181903
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/11
40/40 [==============================] - 8s 132ms/step - loss: 1.0897 - x_prob_reshape_loss: 0.9548 - x_boxes_reshape_loss: 0.1349 - x_prob_reshape_accuracy: 0.7584
Epoch 2/11
40/40 [==============================] - 5s 130ms/step - loss: 0.5583 - x_prob_reshape_loss: 0.4723 - x_boxes_reshape_loss: 0.0860 - x_prob_reshape_accuracy: 0.7698
Epoch 3/11
40/40 [==============================] - 5s 131ms/step - loss: 0.5213 - x_prob_reshape_loss: 0.4449 - x_boxes_reshape_loss: 0.0764 - x_prob_reshape_accuracy: 0.7788
Epoch 4/11
40/40 [==============================] - 5s 128ms/step - loss: 0.5150 - x_prob_reshape_loss: 0.4409 - x_boxes_reshape_loss: 0.0740 - x_prob_reshape_accuracy: 0.7810
Epoch 5/11
40/40 [==============================] - 5s 129ms/step - loss: 0.5233 - x_prob_reshape_loss: 0.4469 - x_boxes_reshape_loss: 0.0764 - x_prob_reshape_accuracy: 0.7771
Epoch 6/11
40/40 [==============================] - 5s 129ms/step - loss: 0.5152 - x_prob_reshape_loss: 0.4411 - x_boxes

/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


epoch,▁▂▂▃▄▅▅▆▇▇█
loss,█▂▁▁▁▁▁▁▁▁▁
x_boxes_reshape_loss,█▃▂▁▂▁▁▁▁▁▁
x_prob_reshape_accuracy,▁▄▇▇▆▇█████
x_prob_reshape_loss,█▁▁▁▁▁▁▁▁▁▁
epoch,10
loss,0.51007
x_boxes_reshape_loss,0.07209
x_prob_reshape_accuracy,0.78286
x_prob_reshape_loss,0.43798


...

epoch,▁▂▂▃▄▅▅▆▇▇█
loss,▃▁▁▁▁▁▁▂▁▂█
x_boxes_reshape_loss,▃▁▁▂▂▂▂▂▂▃█
x_prob_reshape_accuracy,██▇▆▆▆▆▆▆▆▁
x_prob_reshape_loss,▄▁▁▁▁▁▁▁▁▁█
epoch,10
loss,0.84238
x_boxes_reshape_loss,0.11924
x_prob_reshape_accuracy,0.74757
x_prob_reshape_loss,0.72314


wandb: Agent Starting Run: nws1arm2 with config:
wandb: 	batch_size: 256
wandb: 	epochs: 11
wandb: 	learning_rate: 0.043814348862147634
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/11
40/40 [==============================] - 7s 134ms/step - loss: 0.5301 - x_prob_reshape_loss: 0.3678 - x_boxes_reshape_loss: 0.1624 - x_prob_reshape_accuracy: 0.8475
Epoch 2/11
40/40 [==============================] - 5s 134ms/step - loss: 0.2409 - x_prob_reshape_loss: 0.1519 - x_boxes_reshape_loss: 0.0890 - x_prob_reshape_accuracy: 0.9515
Epoch 3/11
40/40 [==============================] - 5s 134ms/step - loss: 0.2018 - x_prob_reshape_loss: 0.1350 - x_boxes_reshape_loss: 0.0669 - x_prob_reshape_accuracy: 0.9502
Epoch 4/11
40/40 [==============================] - 5s 131ms/step - loss: 0.1846 - x_prob_reshape_loss: 0.1257 - x_boxes_reshape_loss: 0.0588 - x_prob_reshape_accuracy: 0.9525
Epoch 5/11
40/40 [==============================] - 5s 133ms/step - loss: 0.1620 - x_prob_reshape_loss: 0.1064 - x_boxes_reshape_loss: 0.0555 - x_prob_reshape_accuracy: 0.9609
Epoch 6/11
40/40 [==============================] - 5s 132ms/step - loss: 0.1576 - x_prob_reshape_loss: 0.1036 - x_boxes

/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


epoch,▁▂▂▃▄▅▅▆▇▇█
loss,█▃▂▂▁▁▂▁▁▁▁
x_boxes_reshape_loss,█▃▂▂▁▁▁▁▁▁▁
x_prob_reshape_accuracy,▁▇▇▇██▇████
x_prob_reshape_loss,█▃▂▂▂▁▂▁▁▁▁
epoch,10
loss,0.13389
x_boxes_reshape_loss,0.04981
x_prob_reshape_accuracy,0.96753
x_prob_reshape_loss,0.08407
